In [2]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

     |████████████████████████████████| 25.0MB 1.7MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


     |████████████████████████████████| 30.6MB 1.5MB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [0]:
def rotate(image, angle = 90):
  h,w =image.shape[:2]
  (cX, cY) = (w//2, h//2)
  M = cv2.getRotationMatrix2D((cX,cY), angle, 1.0)
  cos = np.abs(M[0,0])
  sin = np.abs(M[0,1])

  #compute new bounding dims of img
  nW = int((h*sin)+(w*cos))
  nH = int((h*cos)+(w*sin))

  #adjust rotation matrix to take into account transl;ation
  M[0,2] +=(nW/2)-cX
  M[1,2] +=(nH/2)-cY

  return cv2.warpAffine(image, M, (nW,nH))

In [0]:
def affine(src, angle = -50, scale = 0.9):
  srcTri = np.array( [[0, 0], [src.shape[1] - 1, 0], [0, src.shape[0] - 1]] ).astype(np.float32)
  dstTri = np.array( [[0, src.shape[1]*0.33], [src.shape[1]*0.85, src.shape[0]*0.25], [src.shape[1]*0.15, src.shape[0]*0.7]] ).astype(np.float32)
  warp_mat = cv2.getAffineTransform(srcTri, dstTri)
  warp_dst = cv2.warpAffine(src, warp_mat, (src.shape[1], src.shape[0]))
  # Rotating the image after Warp
  center = (warp_dst.shape[1]//2, warp_dst.shape[0]//2)
  angle = -50
  scale = 0.9
  rot_mat = cv2.getRotationMatrix2D( center, angle, scale )
  warp_rotate_dst = cv2.warpAffine(warp_dst, rot_mat, (warp_dst.shape[1], warp_dst.shape[0]))
  #plt.imshow(src)
  #plt.imshow(warp_dst)
  #plt.imshow(warp_rotate_dst)
  return warp_rotate_dst

In [0]:
def sift(original_img):
  sift = cv2.xfeatures2d.SIFT_create()
  kp, des = sift.detectAndCompute(original_img,None)
  return (kp, des)

In [0]:
def draw_keypoints(kp, img):
  sift_img = cv2.drawKeypoints(img, kp, np.array([]), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  return sift_img

In [0]:
def matcher(img1, img2):
  kp1, des1 = sift(img1)
  kp_img1 = draw_keypoints(kp1, img1)

  kp2,des2 = sift(img2)
  kp_img2 = draw_keypoints(kp2, img2)

  # BFMatcher with default params
  bf = cv2.BFMatcher()
  matches = bf.knnMatch(des1, des2, k=2)
  # Apply ratio test
  good = []
  for m,n in matches:
      if m.distance < 0.75*n.distance:
          good.append([m])
  # cv.drawMatchesKnn expects list of lists as matches.
  bf_img = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict(checks=50)   # or pass empty dictionary
  flann = cv2.FlannBasedMatcher(index_params,search_params)
  matches = flann.knnMatch(des1,des2,k=2)
  # Need to draw only good matches, so create a mask
  matchesMask = [[0,0] for i in range(len(matches))]
  # ratio test as per Lowe's paper
  for i,(m,n) in enumerate(matches):
      if m.distance < 0.7*n.distance:
          matchesMask[i]=[1,0]
  draw_params = dict(matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
  flann_img = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches, None, **draw_params)


  fig = plt.figure(figsize=(10,10))
  a = fig.add_subplot(2, 2, 1)
  a.set_title("Original Image")
  imgplot = plt.imshow(img1)

  a = fig.add_subplot(2, 2, 2)
  a.set_title("With Keypoints")
  imgplot = plt.imshow(kp_img1)

  a = fig.add_subplot(2, 2, 3)
  a.set_title("Transformed Image")
  imgplot = plt.imshow(img2)

  a = fig.add_subplot(2, 2, 4)
  a.set_title("With Keypoints")
  imgplot = plt.imshow(kp_img2)

  fig2 = plt.figure(figsize=(10,10))
  a = fig2.add_subplot(1, 1, 1)
  a.set_title("BF Matcher")
  imgplot = plt.imshow(bf_img)

  fig3 = plt.figure(figsize=(10,10))
  a = fig3.add_subplot(1, 1, 1)
  a.set_title("FLANN Matcher")
  imgplot = plt.imshow(flann_img)


In [75]:
img_list = ['football.jpg', 'gantrycrane.png', 'face.jpeg']
for img_path in img_list:
  img_path = '/content/'+img_path
  img = cv2.imread(img_path)
  rotate_img = rotate(img, 30)
  affine_img = affine(img)

  matcher(img, rotate_img)
  matcher(img, affine_img)

Output hidden; open in https://colab.research.google.com to view.